In [1]:
# imports
#test
import pandas as pd

pd.set_option('display.max_columns', None)

/Users/eric/opt/anaconda3/lib/python3.8/site-packages/pandas/core/computation/expressions.py:20: UserWarning: Pandas requires version '2.7.3' or newer of 'numexpr' (version '2.7.1' currently installed).
  from pandas.core.computation.check import NUMEXPR_INSTALLED


## Read in Data

In [2]:
df = pd.read_csv('Data/vehicles.csv')
df.head()

,id,url,region,region_url,price,year,manufacturer,model,condition,cylinders,fuel,odometer,title_status,transmission,VIN,drive,size,type,paint_color,image_url,description,county,state,lat,long,posting_date
0,7222695916,https://prescott.craigslist.org/cto/d/prescott...,prescott,https://prescott.craigslist.org,6000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,az,NaN,NaN,NaN
1,7218891961,https://fayar.craigslist.org/ctd/d/bentonville...,fayetteville,https://fayar.craigslist.org,11900,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,ar,NaN,NaN,NaN
2,7221797935,https://keys.craigslist.org/cto/d/summerland-k...,florida keys,https://keys.craigslist.org,21000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,fl,NaN,NaN,NaN
3,7222270760,https://worcester.craigslist.org/cto/d/west-br...,worcester / central MA,https://worcester.craigslist.org,1500,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,ma,NaN,NaN,NaN
4,7210384030,https://greensboro.craigslist.org/cto/d/trinit...,greensboro,https://greensboro.craigslist.org,4900,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,nc,NaN,NaN,NaN


In [3]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 426880 entries, 0 to 426879
Data columns (total 26 columns):
 #   Column        Non-Null Count   Dtype  
---  ------        --------------   -----  
 0   id            426880 non-null  int64  
 1   url           426880 non-null  object 
 2   region        426880 non-null  object 
 3   region_url    426880 non-null  object 
 4   price         426880 non-null  int64  
 5   year          425675 non-null  float64
 6   manufacturer  409234 non-null  object 
 7   model         421603 non-null  object 
 8   condition     252776 non-null  object 
 9   cylinders     249202 non-null  object 
 10  fuel          423867 non-null  object 
 11  odometer      422480 non-null  float64
 12  title_status  418638 non-null  object 
 13  transmission  424324 non-null  object 
 14  VIN           265838 non-null  object 
 15  drive         296313 non-null  object 
 16  size          120519 non-null  object 
 17  type          334022 non-null  object 
 18  pain

In [4]:
# calculate missing values percentage per column
round(df.isnull().mean() * 100, 2)

id                0.00
url               0.00
region            0.00
region_url        0.00
price             0.00
year              0.28
manufacturer      4.13
model             1.24
condition        40.79
cylinders        41.62
fuel              0.71
odometer          1.03
title_status      1.93
transmission      0.60
VIN              37.73
drive            30.59
size             71.77
type             21.75
paint_color      30.50
image_url         0.02
description       0.02
county          100.00
state             0.00
lat               1.53
long              1.53
posting_date      0.02
dtype: float64

## Initial Data Cleaning
- Remove `county`, contains no actual values of use.
- Keep relevant columns and remove the additional following columns: `id`, `url`, `region_url`, `VIN`, `image_url`, `lat`, `long`.
- Rename `year` as `year_manufactured`.
* Will need enough information per data entry for the model to make predictions. Drop all values where the following are missing since it will be difficult to impute these values: 
    - `manufacturer`, `model`, `odometer`
    
* Will need to impute majority of missing values; **NOTE: The imputing process will be investigated AFTER the train-test split to avoid data leakage. See Model Building notebook.**  
    - KNN imputer: `condition`, `cylinders`
    - Assess each unique value and see if we can determine appropriate imputed value based on other car info: `drive`, `size`, `type`, and `paint_color`

Feature Engineering Notes:
- Year of selling date can be extracted from `posting_date` 

### Removing/Renaming Columns & Dropping Rows with Missing Values

In [5]:
# keep relevant columns
cols_to_remove = ['county', 'id', 'url', 'region_url', 'VIN', 'image_url', 'lat', 'long', 'description']
# drop cols
vehicles_df = df.drop(columns = cols_to_remove)

# drop rows where data is missing in the following columns
missing_cols = ['manufacturer', 'model', 'odometer']
vehicles_df = vehicles_df.dropna(subset = missing_cols)

# rename year col
vehicles_df = vehicles_df.rename(columns = {'year':'year_manufactured',
                                            'odometer': 'miles'})
vehicles_df

,region,price,year_manufactured,manufacturer,model,condition,cylinders,fuel,miles,title_status,transmission,drive,size,type,paint_color,state,posting_date
27,auburn,33590,2014.0,gmc,sierra 1500 crew cab slt,good,8 cylinders,gas,57923.0,clean,other,NaN,NaN,pickup,white,al,2021-05-04T12:31:18-0500
28,auburn,22590,2010.0,chevrolet,silverado 1500,good,8 cylinders,gas,71229.0,clean,other,NaN,NaN,pickup,blue,al,2021-05-04T12:31:08-0500
29,auburn,39590,2020.0,chevrolet,silverado 1500 crew,good,8 cylinders,gas,19160.0,clean,other,NaN,NaN,pickup,red,al,2021-05-04T12:31:25-0500
30,auburn,30990,2017.0,toyota,tundra double cab sr,good,8 cylinders,gas,41124.0,clean,other,NaN,NaN,pickup,red,al,2021-05-04T10:41:31-0500
31,auburn,15000,2013.0,ford,f-150 xlt,excellent,6 cylinders,gas,128000.0,clean,automatic,rwd,full-size,truck,black,al,2021-05-03T14:02:03-0500
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
426875,wyoming,23590,2019.0,nissan,maxima s sedan 4d,good,6 cylinders,gas,32226.0,clean,other,fwd,NaN,sedan,NaN,wy,2021-04-04T03:21:31-0600
426876,wyoming,30590,2020.0,volvo,s60 t5 momentum sedan 4d,good,NaN,gas,12029.0,clean,other,fwd,NaN,sedan,red,wy,2021-04-04T03:21:29-0600
426877,wyoming,34990,2020.0,cadillac,xt4 sport suv 4d,good,NaN,diesel,4174.0,clean,other,NaN,NaN,hatchback,white,wy,2021-04-04T03:21:17-0600
426878,wyoming,28990,2018.0,lexus,es 350 sedan 4d,good,6 cylinders,gas,30112.0,clean,other,fwd,NaN,sedan,silver,wy,2021-04-04T03:21:11-0600


### Feature Engineer `year_manufactured`

In [6]:
# extract year from posting_date and append posting_year
vehicles_df['posting_year'] = vehicles_df['posting_date'].astype(str).str[0:4]

# drop posting date column
vehicles_df = vehicles_df.drop(columns = 'posting_date')

# reorder columns
vehicles_df = vehicles_df[['posting_year', 'year_manufactured', 'manufacturer', 'model', 'state', 'region', 'price',
                           'fuel', 'miles', 'cylinders', 'condition', 'title_status', 'transmission',
                           'drive', 'size', 'type', 'paint_color']]

vehicles_df.head(3)

,posting_year,year_manufactured,manufacturer,model,state,region,price,fuel,miles,cylinders,condition,title_status,transmission,drive,size,type,paint_color
27,2021,2014.0,gmc,sierra 1500 crew cab slt,al,auburn,33590,gas,57923.0,8 cylinders,good,clean,other,NaN,NaN,pickup,white
28,2021,2010.0,chevrolet,silverado 1500,al,auburn,22590,gas,71229.0,8 cylinders,good,clean,other,NaN,NaN,pickup,blue
29,2021,2020.0,chevrolet,silverado 1500 crew,al,auburn,39590,gas,19160.0,8 cylinders,good,clean,other,NaN,NaN,pickup,red


Additional cleaning to remove missing values in `year_manufactured`.

In [7]:
# drop missing year_manufactured
vehicles_df = vehicles_df.dropna(subset = 'year_manufactured')

# convert to int type
vehicles_df['year_manufactured'] = vehicles_df['year_manufactured'].astype(int)

### Recheck Dataframe for Missing Values

In [8]:
vehicles_df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 399883 entries, 27 to 426879
Data columns (total 17 columns):
 #   Column             Non-Null Count   Dtype  
---  ------             --------------   -----  
 0   posting_year       399883 non-null  object 
 1   year_manufactured  399883 non-null  int64  
 2   manufacturer       399883 non-null  object 
 3   model              399883 non-null  object 
 4   state              399883 non-null  object 
 5   region             399883 non-null  object 
 6   price              399883 non-null  int64  
 7   fuel               397874 non-null  object 
 8   miles              399883 non-null  float64
 9   cylinders          235585 non-null  object 
 10  condition          237800 non-null  object 
 11  title_status       392710 non-null  object 
 12  transmission       398275 non-null  object 
 13  drive              280317 non-null  object 
 14  size               112695 non-null  object 
 15  type               316647 non-null  object 
 16  paint_

In [9]:
round(vehicles_df.isna().mean() * 100, 2)

posting_year          0.00
year_manufactured     0.00
manufacturer          0.00
model                 0.00
state                 0.00
region                0.00
price                 0.00
fuel                  0.50
miles                 0.00
cylinders            41.09
condition            40.53
title_status          1.79
transmission          0.40
drive                29.90
size                 71.82
type                 20.82
paint_color          29.84
dtype: float64

## Additional Cleaning
There are still many missing values in the dataset. To further simply the dataset, lets focus on the available vehicles in `model`.
- We will limit the dataset to contain only car models where a unique value for `model` appears more than 10 times in the dataset. Upon inspection, there are very specific model types, typos, or variations of car models that will not be useful for the purpose of this analysis. 

In [10]:
# get all the unique models
model_count = vehicles_df['model'].value_counts()
print(model_count[model_count >10])

model
f-150                                    7870
silverado 1500                           5043
1500                                     4177
camry                                    3070
silverado                                2989
                                         ... 
civic coupe lx                             11
f-350 sd lariat crew cab long bed 4wd      11
express commercial                         11
cl                                         11
juke nismo                                 11
Name: count, Length: 3852, dtype: int64


In [11]:
# get all rows where unique models appear more than 10 times
vc = vehicles_df['model'].value_counts() > 10
vc[vc]

model
f-150                                    True
silverado 1500                           True
1500                                     True
camry                                    True
silverado                                True
                                         ... 
civic coupe lx                           True
f-350 sd lariat crew cab long bed 4wd    True
express commercial                       True
cl                                       True
juke nismo                               True
Name: count, Length: 3852, dtype: bool

In [12]:
# filter dataframe for unique models appearing more than 10 times
vehicles_df = vehicles_df.loc[vehicles_df['model'].isin(vc.index)].reset_index(drop = True)
vehicles_df

,posting_year,year_manufactured,manufacturer,model,state,region,price,fuel,miles,cylinders,condition,title_status,transmission,drive,size,type,paint_color
0,2021,2014,gmc,sierra 1500 crew cab slt,al,auburn,33590,gas,57923.0,8 cylinders,good,clean,other,NaN,NaN,pickup,white
1,2021,2010,chevrolet,silverado 1500,al,auburn,22590,gas,71229.0,8 cylinders,good,clean,other,NaN,NaN,pickup,blue
2,2021,2020,chevrolet,silverado 1500 crew,al,auburn,39590,gas,19160.0,8 cylinders,good,clean,other,NaN,NaN,pickup,red
3,2021,2017,toyota,tundra double cab sr,al,auburn,30990,gas,41124.0,8 cylinders,good,clean,other,NaN,NaN,pickup,red
4,2021,2013,ford,f-150 xlt,al,auburn,15000,gas,128000.0,6 cylinders,excellent,clean,automatic,rwd,full-size,truck,black
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
399878,2021,2019,nissan,maxima s sedan 4d,wy,wyoming,23590,gas,32226.0,6 cylinders,good,clean,other,fwd,NaN,sedan,NaN
399879,2021,2020,volvo,s60 t5 momentum sedan 4d,wy,wyoming,30590,gas,12029.0,NaN,good,clean,other,fwd,NaN,sedan,red
399880,2021,2020,cadillac,xt4 sport suv 4d,wy,wyoming,34990,diesel,4174.0,NaN,good,clean,other,NaN,NaN,hatchback,white
399881,2021,2018,lexus,es 350 sedan 4d,wy,wyoming,28990,gas,30112.0,6 cylinders,good,clean,other,fwd,NaN,sedan,silver


There are still many missing values that will be unable to be imputed reasonably. 

For example, `size` has about 70% missing values and is a dimension specific measurement that is based on the interior size of the vehicle. Similarly, `condition` is a subjective measurement that contains missing values that would be impossible to determine just based on the data alone. 

We will make the choice to drop these subjective categorical features for the above reason and simplify model complexity.

In [13]:
# drop size and condition cols
vehicles_df = vehicles_df.drop(columns = ['size','condition'])

In [14]:
# extract year from posting_date and append posting_year
vehicles_df['posting_year'] = vehicles_df['posting_year'].astype(int)

In [15]:
# copy cleaned dataframe
cleaned_vehicles_df = vehicles_df.copy()
cleaned_vehicles_df.head()

,posting_year,year_manufactured,manufacturer,model,state,region,price,fuel,miles,cylinders,title_status,transmission,drive,type,paint_color
0,2021,2014,gmc,sierra 1500 crew cab slt,al,auburn,33590,gas,57923.0,8 cylinders,clean,other,NaN,pickup,white
1,2021,2010,chevrolet,silverado 1500,al,auburn,22590,gas,71229.0,8 cylinders,clean,other,NaN,pickup,blue
2,2021,2020,chevrolet,silverado 1500 crew,al,auburn,39590,gas,19160.0,8 cylinders,clean,other,NaN,pickup,red
3,2021,2017,toyota,tundra double cab sr,al,auburn,30990,gas,41124.0,8 cylinders,clean,other,NaN,pickup,red
4,2021,2013,ford,f-150 xlt,al,auburn,15000,gas,128000.0,6 cylinders,clean,automatic,rwd,truck,black


In [16]:
cleaned_vehicles_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 399883 entries, 0 to 399882
Data columns (total 15 columns):
 #   Column             Non-Null Count   Dtype  
---  ------             --------------   -----  
 0   posting_year       399883 non-null  int64  
 1   year_manufactured  399883 non-null  int64  
 2   manufacturer       399883 non-null  object 
 3   model              399883 non-null  object 
 4   state              399883 non-null  object 
 5   region             399883 non-null  object 
 6   price              399883 non-null  int64  
 7   fuel               397874 non-null  object 
 8   miles              399883 non-null  float64
 9   cylinders          235585 non-null  object 
 10  title_status       392710 non-null  object 
 11  transmission       398275 non-null  object 
 12  drive              280317 non-null  object 
 13  type               316647 non-null  object 
 14  paint_color        280565 non-null  object 
dtypes: float64(1), int64(3), object(11)
memory usage: 4

In [17]:
# save as cleaned csv
cleaned_vehicles_df.to_csv('Data/cleaned_vehicles_df.csv')